In [5]:
import torch
from torch import nn
import torch.nn.functional as F

def corr2d(X, K):
    h , w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1 , X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i , j] = (X[i : i + h,j : j + w] * K).sum()
    return Y
#h为行数，w为列数


In [6]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = torch.tensor([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

In [7]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

In [8]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [9]:
K = torch.tensor([[1.0, -1.0]])

In [10]:
Y = corr2d(X, K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [13]:
conv2d = nn.Conv2d(1 , 1 ,(1 , 2) , bias = False)
X = X.reshape((1 , 1 , 6 , 8))#第一个是批量大小，第二个是通道，第三个是长，第四个是宽
Y = Y.reshape((1 , 1 , 6 , 7))
for i in range(10):
    y_hat = conv2d(X)
    l = (y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()
    conv2d.weight.data[:] -= 3e-2 * conv2d.weight.grad #相当于学习率是3e-2，这里相当于梯度下降，因为没有设置迭代器。
    if(i + 1) % 2 == 0:
        print(f'batch{i + 1},loss{l.sum():.3f}')

conv2d.weight.data.reshape((1,-1))

batch2,loss1.473
batch4,loss0.320
batch6,loss0.083
batch8,loss0.026
batch10,loss0.009


tensor([[ 0.9849, -1.0039]])